In [1]:
import sys
import os
import warnings
import numpy as np
import tensorflow as tf
import SimpleITK as sitk
from nilearn import image, plotting
import nibabel as nib
import matplotlib.pyplot as plt

sys.path.append("../")
sys.path.append("../deepneuroan")
sys.path.append('/home/ltetrel/DeepNeuroAN/deepneuroan/')
sys.path.append('/home/ltetrel/DeepNeuroAN/')
warnings.filterwarnings('ignore')

from deepneuroan.data_generator import DataGenerator
from deepneuroan.models import ChannelwiseConv3D, rigid_concatenated
import deepneuroan.utils as utils
from deepneuroan.preproc import create_ref_grid, get_mni152_affine_nib, get_mni152_affine_itk

Failed to import duecredit due to No module named 'duecredit'


In [35]:
data_dir = "/home/ltetrel/Documents/data/neuromod/derivatives/deepneuroan/training/generated_data/"
template_filepath = os.path.join(data_dir, "template_on_grid")

list_files = []
list_files_tmp = set([])
for root, _, files in os.walk(data_dir):
    for file in files:
        filepath = os.path.join(root, file).split('.')[0]
        if os.path.exists(filepath + ".txt"):
            list_files_tmp.add(filepath)
list_files = list(list_files_tmp)

bs = 3
ncpu = 1
np.random.seed(0)
params_gen = dict(list_files=list_files, template_file=template_filepath, batch_size=bs, avail_cores=ncpu)
gen = DataGenerator(partition="train", **params_gen)

In [98]:
ref_grid = create_ref_grid()

# tmp
batch = 0

moving = gen.__getitem__(2)[0][0][batch, :, :, :,0]
fixed = gen.__getitem__(2)[0][1][batch, :, :, :,0]
print(gen.get_files_batch(0))
greater = tf.cast(tf.greater(moving, 0.), tf.float32)
center = tf.reduce_mean(tf.where(greater), axis=0)
print(ref_grid.TransformIndexToPhysicalPoint(center.numpy().tolist()))

['/home/ltetrel/Documents/data/neuromod/derivatives/deepneuroan/training/generated_data/ses-vid001_task-video_run-01_bold_vol-0127_transfo-000001', '/home/ltetrel/Documents/data/neuromod/derivatives/deepneuroan/training/generated_data/ses-vid001_task-video_run-01_bold_vol-0127_transfo-000002', '/home/ltetrel/Documents/data/neuromod/derivatives/deepneuroan/training/generated_data/ses-vid001_task-video_run-01_bold_vol-0127_transfo-000003']
(-10.0, 22.0, 17.0)


In [ ]:
sitk.WriteImage(registered_moving, "registered_moving.nii.gz")
sitk.WriteImage(sitk.ReadImage(valid_gen.get_files_batch(0)[batch] + ".nii.gz", sitk.sitkFloat32), "fixed.nii.gz")

In [ ]:
# def dice_loss(y_true, y_pred):
  
#     num = 2 * tf.reduce_sum(y_true * y_pred, -1)
#     den = tf.maximum(tf.reduce_sum(y_true + y_pred, -1), 1e-6)

#     return -tf.reduce_mean(num/den)

def dice_loss(y_true, y_pred):
    
    true_masked = tf.clip_by_value(y_true*1e9, 0, 1)
    pred_masked = tf.clip_by_value(y_pred*1e9, 0, 1)

    numerator = 2 * tf.reduce_sum(true_masked * pred_masked)
    denominator = tf.reduce_sum(true_masked) + tf.reduce_sum(pred_masked)    
    return (numerator + 1) / (denominator + 1)

def dice_loss_mask(y_true, y_pred):
  
    numerator = 2 * np.sum(y_true * y_pred)
    denominator = np.sum(y_true) + np.sum(y_pred)    
    return (numerator + 1) / (denominator + 1)

print(dice_loss_mask(truth_mask, pred_mask))
print(dice_loss(fixed, sitk.GetArrayFromImage(registered_moving)))

In [ ]:
background = nib.Nifti1Image(fixed, get_mni152_affine_nib())
edges = nib.Nifti1Image(sitk.GetArrayFromImage(registered_moving), get_mni152_affine_nib())

display = plotting.plot_anat(background)
display.add_edges(edges)

In [ ]:
nib_img = nib.load("/home/ltetrel/Documents/data/neuromod/derivatives/deepneuroan/training/generated_data/ses-vid001_task-video_run-01_bold_vol-0127.nii.gz")
print(nib_img.get_affine()
itk_img = sitk.ReadImage("/home/ltetrel/Documents/data/neuromod/derivatives/deepneuroan/training/generated_data/ses-vid001_task-video_run-01_bold_vol-0127.nii.gz")
print(itk_img.GetDirection())
print(itk_img.GetSpacing())
print(itk_img.GetOrigin())
print(itk_img.GetSize())

In [ ]:
itk_tmp = sitk.GetImageFromArray(itk_data)
itk_tmp.SetDirection(get_mni152_affine_itk()[:3, :3].flatten())
itk_tmp.SetSpacing((1., 1., 1.))
itk_tmp.SetOrigin(get_mni152_affine_itk()[:3,3])
sitk.WriteImage(itk_tmp, "itk_tmp.nii.gz")

nib_tmp = sitk.GetImageFromArray(nib_data)
# nib_tmp.SetDirection(get_mni152_affine_nib()[:3, :3].flatten())
nib_tmp.SetSpacing((1., 1., 1.))
nib_tmp.SetOrigin(get_mni152_affine_nib()[:3,3])
sitk.WriteImage(nib_tmp, "nib_tmp.nii.gz")

In [ ]:
nib_data = nib_img.get_data()
itk_data = sitk.GetArrayFromImage(itk_img)
# recreated_sitk = sitk.GetImageFromArray(nib_data)
# recreated_sitk.SetOrigin(ref_grid.GetOrigin())
# recreated_sitk.SetSpacing(ref_grid.GetSpacing())
# recreated_sitk.SetDirection(ref_grid.GetDirection())
# itk_data = sitk.GetArrayFromImage(recreated_sitk)
mask = nib_data != itk_data
print(np.sum(mask)/(nib_data.shape[0] * nib_data.shape[1] * nib_data.shape[2]))